In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
import tensorflow as tf

%matplotlib notebook

from IPython.display import display, HTML

pd.set_option('display.max_columns', None)

from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [127]:
data = pd.read_csv(
    "Data/train_values.csv", 
    index_col='row_id', 
    parse_dates=['timestamp'], nrows = 5 * 10 ** 5
)

# Data Preparation

1. Exclude `final_rinse` data
2. Sample according to the phase distributions
3. Separate out `process_id`. We don't want to encode or scale this information. We'll just reattach after encoding and scaling the rest.

In [128]:
data = data[data.phase != 'final_rinse']

In [129]:
pids = data.process_id.unique()
phase_categorical = CategoricalDtype(
    categories=['pre_rinse', 'caustic', 'intermediate_rinse', 'acid', 'final_rinse'],
    ordered=True
)

In [130]:
shuffled_idx = np.random.permutation(np.arange(len(pids)))
n = len(shuffled_idx)

phase_pid_idx = {
    'pre_rinse': shuffled_idx[:int(n * 0.1)],
    'caustic': shuffled_idx[int(n * 0.1): int(n * 0.4)],
    'intermediate_rinse': shuffled_idx[int(n * 0.4): int(n * 0.7)],
    'acid': shuffled_idx[int(n * 0.7):]
}
# pid: phase_cutoff
# pid = pids[index]
pid2phase_list = []
for phase in phase_pid_idx:
    cur_pids = pids[phase_pid_idx[phase]]
    pid2phase_list.extend(list(zip(cur_pids, np.repeat(phase, len(cur_pids)))))

In [131]:
pid2phase_cutoff = dict(pid2phase_list)

In [132]:
max_length = 0
for pid in pids:
    cur_length = data[data.process_id == pid].shape[0]
    if cur_length > max_length:
        max_length = cur_length

In [133]:
max_length

2753

In [134]:
data.columns

Index(['process_id', 'object_id', 'phase', 'timestamp', 'pipeline',
       'supply_flow', 'supply_pressure', 'return_temperature',
       'return_conductivity', 'return_turbidity', 'return_flow', 'supply_pump',
       'supply_pre_rinse', 'supply_caustic', 'return_caustic', 'supply_acid',
       'return_acid', 'supply_clean_water', 'return_recovery_water',
       'return_drain', 'object_low_level', 'tank_level_pre_rinse',
       'tank_level_caustic', 'tank_level_acid', 'tank_level_clean_water',
       'tank_temperature_pre_rinse', 'tank_temperature_caustic',
       'tank_temperature_acid', 'tank_concentration_caustic',
       'tank_concentration_acid', 'tank_lsh_caustic', 'tank_lsh_acid',
       'tank_lsh_clean_water', 'tank_lsh_pre_rinse', 'target_time_period'],
      dtype='object')

* Let's separate out `process_id`. We don't want to encode or scale this information. We'll just reattach after encoding and scaling the rest.

In [135]:
process_id_array = data.process_id.values
data.drop(['process_id', 'object_id'], axis=1, inplace=True)

# Data Encoding

Let's figure out how to preprocess the data into RNN-feedable form.

* `phase` will NOT be one-hot-encoded since there is an order to it, so we can simply encode it using 1, 2, .., 5. 
  * Using `data.phase.factorize()` renders the phases to be 0, 1, ... so let's add 1 to all after that line.
  * We don't want processes that end in `pre_rinse` stage to be lost when masking 0s.. 

In [136]:
phase_categorical = CategoricalDtype(
    categories=['pre_rinse', 'caustic', 'intermediate_rinse', 'acid', 'final_rinse'],
    ordered=True
)
data.phase = data.phase.astype(phase_categorical)

data.phase, phase_mapping_idx = data.phase.factorize()
data.phase = data.phase + 1

In [137]:
phase_mapping_idx.categories

Index(['pre_rinse', 'caustic', 'intermediate_rinse', 'acid', 'final_rinse'], dtype='object')

* Ensure each sequence is in the right order using the `timestamp` column, but once sequences are set up, discard the column.
  * The column is only useful insofar as it tells us which data point comes before or after others.
  * But then again, **perhaps there is some signal from absoluate passage of time, so we can consider encoding this into UNIX times.** 
 Something to try out.  

In [138]:
data.timestamp = data.timestamp.view(int)

* Ensure each sequence is from single process using `process_id`, but don't include it in data since it really is just an ID column.

* One-hot encode `object_id` and `pipeline`.
  * Although `object_id` is an ID column, there aren't that many objects (~100? **NOT SURE**) and there are multiple processes over each object. So `object_id` may carry valuable information.
  * I'm not sure what `pipeline` is, but we'll treat it as a categorical column
  * Below we first encode categorical columns into integers (and save that mapping in DataframeLabelEncoder.feature_encoder dictionary) and one-hot-encode them. (sklearn's OneHotEncoder requires that input is alreay integer-encoded.)

In [139]:
categorical_features = ['pipeline']

In [140]:
from sklearn.preprocessing import LabelEncoder


class DataframeLabelEncoder(object):
    def __init__(self, categorical_features):
        assert isinstance(categorical_features, (list, np.ndarray))
        self.categorical_features = categorical_features
        self.feature_encoder = dict()
        self.cat_feature_mask = None
        
    def fit(self, dataframe):
        assert isinstance(dataframe, pd.DataFrame)
        
        if self.cat_feature_mask is None:
            self.cat_feature_mask = np.zeros(shape=dataframe.shape[1], dtype=bool)
        
        for i, feature in enumerate(categorical_features):
            le = LabelEncoder()
            le.fit(dataframe[feature])
            self.feature_encoder[feature] = le
            
            index = np.where(dataframe.columns == feature)[0][0]
            self.cat_feature_mask[index] = True
        
    def transform(self, dataframe):
        assert dataframe.shape[1] == self.cat_feature_mask.shape[-1]
        
        array = dataframe.values.copy()
        for feature in categorical_features:
            encoded = self.feature_encoder[feature].transform(
                dataframe[feature]
            )
            index = np.where(dataframe.columns == feature)[0][0]
            array[:, index] = encoded
            
        return array
    
    def fit_transform(self, dataframe):
        self.fit(dataframe)
        return self.transform(dataframe)

In [141]:
label_encoder = DataframeLabelEncoder(categorical_features)
label_encoder.fit(data)
encoded_data = label_encoder.transform(data)

In [142]:
encoded_data.shape

(373587, 33)

In [143]:
# Create One-hot encoder
ohe = OneHotEncoder(categorical_features=label_encoder.cat_feature_mask, sparse=False)

In [144]:
encoded_data = ohe.fit_transform(encoded_data)

Check that is the expected shape: # non categorical features (= #num features - # num categorical features) + # unique objects + # unique pipelines

In [145]:
encoded_data.shape

(373587, 43)

In [146]:
data.pipeline.nunique()

11

In [147]:
N_ENCODED_FEATURES = encoded_data.shape[1]

## Scaling data

From EDA, it looks like some are normally distributed, some exponential... we'll just standardize.

I thought that we should only standardize columns that were originally numerical (and not boolean or categorical), but it's too much work.. I think just standardizing everything is fine!!

In [148]:
labels = pd.read_csv("Data/train_labels.csv")

In [149]:
preprocessor = Pipeline(
    steps=[
        ('label_encoder', label_encoder),
        ('one_hot_encoder', ohe),
        ('scaler', StandardScaler().fit(encoded_data))
    ]
)

NOW THAT's OUR PREPROCESSOR!!! HOORAY!

Ok let's just create the padded array now. (What Holden did!) 

From our EDA, we know the maximum length of a process is **15107**.
We also know there are 34 columns (excluding process id), but that gets encoded to **84 columns.** Following Holdens' work, 

> (m, T, f) - where m is the number of unique processes, T is the number of time-sequences, and f is the number of features

Our final array shall have shape $(82, 15107, 84)$

**Note that we've only loaded first 100K rows. Update this part if you're using your own sampled data or the whole dataset!! # of unique process ids, max lengt will be different.**

In [150]:
(process_id_array == pid).shape

(373587,)

Before we transform our data, let's drop unnecessary ones according to our `phase_pid_idx` dictionary.

In [151]:
encoded_data.shape

(373587, 43)

In [152]:
n

407

Let's cut them off!

In [153]:
def get_phase_idx(phase):
    phase_index = np.where(
        phase_categorical.categories == phase
    )[0][0]
    return phase_index + 1

In [154]:
arr = np.zeros((len(pids), 3434, 43))
for i, pid in enumerate(pids):
    pid_mask = process_id_array == pid  # mask for this process id
    
    # Cutoff up to appropriate phase
    phase_idx = get_phase_idx(pid2phase_cutoff[pid])
    pid_data = data[pid_mask]
    
    truncated_data = pid_data[pid_data.phase <= phase_idx]
    if truncated_data.shape[0] == 0:
        continue
    
    preprocessed_data = preprocessor.transform(truncated_data)
    
    nrows = preprocessed_data.shape[0]
    arr[i, :nrows, :] = preprocessed_data


In [155]:
empty_process_mask = arr[:, 0].sum(axis=1) == 0  # To get rid of process ids for which the phase cutoff got rid of everythinG@@1

In [156]:
arr = arr[~empty_process_mask]

In [157]:
arr = np.float32(arr)

# `arr` is our final data :)

## Prep labels

In [158]:
arr.shape, labels.shape

((395, 3434, 43), (5021, 2))

In [159]:
labels_mask = np.zeros(labels.shape[0], dtype=bool)
final_pids = pids[~empty_process_mask]
for i, pid in enumerate(labels.process_id):
    if pid in final_pids:
        labels_mask[i] = 1

In [160]:
y = labels.final_rinse_total_turbidity_liter[labels_mask].values

# Can ignore this part ------------- I was thinking........weeks ago... maybe helpful?? idk..
## Reshaping data

But first, we need to figure out how we're going to approach this problem.

Key things to consider:

* **Sequence length**: the processes have different lengths
  * Not only that, the longest process has 15107 data points---very long!!
* **Prediction output**: for each process (=sequence), predict **one** value.

### Turning variable-length sequences into fixed-length?
Say our lookback window size is $L$. We want to reshape data so that each process is expressed in $n$ sequences of length $L$ where $n= \text{length of process} - L + 1$. 

At the end of each process, i.e. at $n$th sequence, the RNN's state must be refreshed.

It is probably the best to create an iterator rather than making and actual array of all of this. Some iterator that managers each process and lookback length as well as signal to refresh RNN states.

Following suggestions from https://danijar.com/variable-sequence-lengths-in-tensorflow/

In [83]:
def length(sequence):
  used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  return length

In [84]:
def batch_iterator(dataX, dataY, batch_size, num_steps):
    data_len = len(dataY)
    batch_len = int(data_len / batch_size)

    if batch_len == 0:
        raise ValueError("epoch_size == 0, decrease batch_size or num_steps")

    for i in range(batch_len):
        input_x = dataX[i * batch_size: (i + 1) * batch_size]
        input_y = dataY[i * batch_size: (i + 1) * batch_size]

        yield (input_x, input_y)

# REAL DEAL

First try with Keras.

First set up config object. This is how we'll keep track of which hyperparms we uesed

In [162]:
class RNNConfig(object):
    def __init__(
        self, cell_type='lstm', window=20, forget_bias=1.0, 
        n_hidden_cells=100, n_layers=1, keep_prob=1.0, batch_size=1, 
        epoch_num=100, learning_rate=0.02, max_grad_norm=1.0, init_scale=0.1,
    ):
        self.cell_type = cell_type
        self.window = window
        self.forget_bias = forget_bias
        self.n_hidden_cells = n_hidden_cells
        self.keep_prob = keep_prob
        self.batch_size = batch_size
        self.epoch_num = epoch_num
        self.learning_rate = learning_rate
        self.max_grad_norm = max_grad_norm
        self.init_scale = init_scale
        self.n_layers = n_layers

A function that creates the model, according to our config input.

In [163]:
def many_to_one_model(config):
    assert isinstance(config, RNNConfig)
    model = tf.keras.models.Sequential()
    
    # Masking layer
    model.add(
        tf.keras.layers.Masking(
            mask_value=0., input_shape=(config.window, N_ENCODED_FEATURES)))
    
    if config.n_layers == 1:
        model.add(
            tf.keras.layers.LSTM(
                config.n_hidden_cells, input_shape=[config.window, N_ENCODED_FEATURES],
                activation='relu'),)
    elif config.n_layers == 2:
        model.add(
            tf.keras.layers.LSTM(
                config.n_hidden_cells, input_shape=[config.window, N_ENCODED_FEATURES], return_sequences=True,
                activation='relu')
        )
        model.add(
            tf.keras.layers.LSTM(
               config.n_hidden_cells, activation='relu'),
        )
    else:
        raise NotImplementedError("Keep n_layers <= 2.")
        
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    loss = 'mean_squared_error'
    rmsprop = tf.keras.optimizers.RMSprop(lr=config.learning_rate)
    model.compile(
        loss=loss, optimizer=rmsprop,
        metrics=[tf.keras.metrics.mean_squared_error]
    )

    return model

### 1. Sanity check that model works

In [164]:
basic_config = RNNConfig(
    window=max_length_test, 
    n_hidden_cells=16, 
    n_layers=1, 
    batch_size=1, 
    epoch_num=1
)

model = many_to_one_model(basic_config)

In [165]:
model.fit(x=arr, y=y, batch_size=basic_config.batch_size, epochs=basic_config.epoch_num, validation_split=0.2)

Train on 316 samples, validate on 79 samples
Epoch 1/1
 10/316 [..............................] - ETA: 41:29 - loss: nan - mean_squared_error: nan                         

KeyboardInterrupt: 

### 2. Now some more complex model!

In [45]:
config_1 = RNNConfig(
    window=max_length, 
    n_hidden_cells=100, 
    n_layers=2, 
    batch_size=1, 
    epoch_num=100
)

model_1 = many_to_one_model(config_1)
model_1.fit(
    x=arr, y=y, 
    batch_size=config_1.batch_size, 
    epochs=config_1.epoch_num, 
    validation_split=0.2
)

Train on 3888 samples, validate on 973 samples
Epoch 1/100
   1/3888 [..............................] - ETA: 70:00:44 - loss: 167971700736.0000 - mean_squared_error: 167971700736.0000

KeyboardInterrupt: 

### Attempt at inspecting test data... for the first time....

In [53]:
test_data = pd.read_csv("Data/test_values.csv", index_col='row_id', 
    parse_dates=['timestamp'],)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


#### 1. permanant changes

In [54]:
test_pids = test_data.process_id.unique()

In [55]:
process_id_array_test = test_data.process_id.values

In [56]:
max_length_test = 0
for pid in test_pids:
    cur_length = test_data[test_data.process_id == pid].shape[0]
    if cur_length > max_length_test:
        max_length_test = cur_length

In [57]:
pid_array_test = test_data.process_id.values
test_data.drop(['process_id', 'object_id'], axis=1, inplace=True)
test_data.phase = test_data.phase.astype(phase_categorical)
test_data.phase, _ = test_data.phase.factorize()
test_data.phase = test_data.phase + 1
test_data.timestamp = test_data.timestamp.view(int)

#### 2. Transform using the preprocessor fit on our training data.

Didn't work because our label encoder couldn't recognize those categories. (probably for new objects..)

I don't think we'll have the same problem if we fit on the entire data, because then we'd know most objects. 

But then again, just in case test set has unseen objects, perhaps we're better off dropping `object_id` altogether?

In [58]:
preprocessed_test = preprocessor.transform(test_data)

#### 3. Reshape so that we have $(\text{processes}, \text{max length}, \text{features})$

In [60]:
test_arr = np.zeros((len(test_pids), max_length_test, 43))
for i, pid in enumerate(test_pids):
    pid_mask = process_id_array_test == pid  # mask for this process id
    pid_data = test_data[pid_mask]
    preprocessed_data = preprocessor.transform(pid_data)
    
    nrows = min(max_length_test, preprocessed_data.shape[0]) 
    # In case some test processes have greater length..
    # nip them at training max_length
    test_arr[i, :nrows, :] = preprocessed_data


### Predictions

In [79]:
yhat = model.predict(test_arr, verbose=1)

2967/2967 [==============================] - 271s 91ms/step


In [80]:
yhat

array([[0.8089831 ],
       [0.99999845],
       [1.        ],
       ...,
       [0.7477091 ],
       [0.99987674],
       [0.5528498 ]], dtype=float32)

# Submission

In [81]:
submission_format = pd.read_csv('Data/submission_format.csv', index_col=0)

In [82]:
my_submission_4 = pd.DataFrame(data=yhat,
                            columns=submission_format.columns,
                            index=submission_format.index)

In [83]:
my_submission_4.to_csv('submission_4.csv')